In [61]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from fake_useragent import UserAgent
from datetime import datetime
import requests
import re
import pandas as pd
import numpy as np
import json

In [65]:
today = datetime.today().strftime('%Y%m%d')
corp_keyword = "삼성전자"

In [66]:
ua = UserAgent()
userAgent = ua.random
print(userAgent)

Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


In [67]:
url_dart_search = r'http://dart.fss.or.kr/dsab001/search.ax'

In [68]:
# "textCrpNm": corp_keyword,
data = [
    ("currentPage", 1),
    ("maxResults", 100),
    ("maxLinks", 10),
    ("sort", "date"),
    ("series", "desc"),
    ("textCrpNm", corp_keyword),
    ("finalReport", "recent"),
    ("startDate", 19990101),
    ("endDate", today),
    ("publicType", "A001"),
    ("publicType", "F002")
]
"""
A001: 사업보고서
A002: 반기보고서
A003: 분기보고서

F001: 감사보고서
F002: 연결감사보고서
F002: 결합감사보고서

"""

'\nA001: 사업보고서\nA002: 반기보고서\nA003: 분기보고서\n\nF001: 감사보고서\nF002: 연결감사보고서\nF002: 결합감사보고서\n\n'

In [69]:
s = requests.Session()
s.headers['User-Agent'] = userAgent

In [70]:
r = s.post(url_dart_search, data)

In [71]:
html = r.text.strip()
soup = BeautifulSoup(html, 'lxml')

In [72]:
soup.find('p',{'class':'page_info'})

<p class="page_info">
		[1/1] [총 35건]
	</p>

In [73]:
target_list = soup.find('table').find_all('tr')
header = target_list[0].text.strip().split('\n')
header.append('c_기업구분')
header.append('c_기업개황_링크')
header.append('c_보고서_링크')
header

['번호',
 '공시대상회사',
 '보고서명',
 '제출인',
 '접수일자',
 '비고',
 'c_기업구분',
 'c_기업개황_링크',
 'c_보고서_링크']

In [75]:
target_list[1].find_all('td')[5]

<td class="cen_txt end"> <img alt="본 보고서는 연결부분을 포함한 것임" hspace="1" src="/images/sub/remark03.gif" title="본 보고서는 연결부분을 포함한 것임"/></td>

In [95]:
result_list = []
result_list.append(header)
for i in range(1,len(target_list)):
    tmp_list = []
    
    item = target_list[i].find_all('td')
    tmp_list.append(item[0].text.strip()) # 번호
    tmp_list.append(item[1].text.strip()) # 공시대상회사
    tmp_list.append(item[2].text.strip().replace('\r','').replace('\n','').replace('\t','')) # 보고서명
    tmp_list.append(item[3].text.strip()) # 제출인
    tmp_list.append(item[4].text.strip()) # 접수일자
    tmp_list.append(item[5].find('img')['title'] if item[5].find('img') is not None else "") # 비고
    tmp_list.append(item[1].find('img')['title']) # c_기업구분
    tmp_list.append('http://dart.fss.or.kr' + item[1].find('a')['href']) # c_기업개황_링크
    tmp_list.append('http://dart.fss.or.kr' + item[2].find('a')['href']) # c_보고서_링크
    
    result_list.append(tmp_list)
    

In [97]:
pd.DataFrame(result_list).head()

,0,1,2,3,4,5,6,7,8
0,번호,공시대상회사,보고서명,제출인,접수일자,비고,c_기업구분,c_기업개황_링크,c_보고서_링크
1,1,삼성전자,사업보고서 (2018.12),삼성전자,2019.04.01,본 보고서는 연결부분을 포함한 것임,유가증권시장,http://dart.fss.or.kr/dsae001/selectPopup.ax?s...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...
2,2,삼성전자,사업보고서 (2017.12),삼성전자,2018.04.02,본 보고서는 연결부분을 포함한 것임,유가증권시장,http://dart.fss.or.kr/dsae001/selectPopup.ax?s...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...
3,3,삼성전자,사업보고서 (2016.12),삼성전자,2017.03.31,본 보고서는 연결부분을 포함한 것임,유가증권시장,http://dart.fss.or.kr/dsae001/selectPopup.ax?s...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...
4,4,삼성전자,사업보고서 (2015.12),삼성전자,2016.03.30,본 보고서는 연결부분을 포함한 것임,유가증권시장,http://dart.fss.or.kr/dsae001/selectPopup.ax?s...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...


In [101]:
df = pd.DataFrame(result_list)
df.columns = df.iloc[0]
df = df[1:]
df.head()

,번호,공시대상회사,보고서명,제출인,접수일자,비고,c_기업구분,c_기업개황_링크,c_보고서_링크
1,1,삼성전자,사업보고서 (2018.12),삼성전자,2019.04.01,본 보고서는 연결부분을 포함한 것임,유가증권시장,http://dart.fss.or.kr/dsae001/selectPopup.ax?s...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...
2,2,삼성전자,사업보고서 (2017.12),삼성전자,2018.04.02,본 보고서는 연결부분을 포함한 것임,유가증권시장,http://dart.fss.or.kr/dsae001/selectPopup.ax?s...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...
3,3,삼성전자,사업보고서 (2016.12),삼성전자,2017.03.31,본 보고서는 연결부분을 포함한 것임,유가증권시장,http://dart.fss.or.kr/dsae001/selectPopup.ax?s...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...
4,4,삼성전자,사업보고서 (2015.12),삼성전자,2016.03.30,본 보고서는 연결부분을 포함한 것임,유가증권시장,http://dart.fss.or.kr/dsae001/selectPopup.ax?s...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...
5,5,삼성전자,연결감사보고서 (2015.12),삼일회계법인,2016.03.14,,유가증권시장,http://dart.fss.or.kr/dsae001/selectPopup.ax?s...,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...


In [85]:
result_list = [i.text.strip() for i in result_list]
result_list

AttributeError: 'list' object has no attribute 'text'

In [63]:
result_list = [i.strip() for i in result_list]
result_list

TypeError: 'NoneType' object is not callable